In [1]:
import cvxpy as cp
import numpy as np

from abacus_simulator.forecaster import Forecaster
from instruments.instruments import Equity

In [2]:
# Create stocks
start = "2005-12-28"
end = "2022-07-11"
interval = "wk"
stock1 = Equity(
    ric="XOM", currency="USD", start_date=start, end_date=end, interval=interval
)
stock2 = Equity(
    ric="CVX", currency="USD", start_date=start, end_date=end, interval=interval
)
stock3 = Equity(
    ric="^GSPC", currency="USD", start_date=start, end_date=end, interval=interval
)
stock4 = Equity(
    ric="WFC", currency="USD", start_date=start, end_date=end, interval=interval
)
stock5 = Equity(
    ric="MSFT", currency="USD", start_date=start, end_date=end, interval=interval
)

In [3]:
instruments = [stock1, stock2, stock3, stock4, stock5]
forc = Forecaster(instruments=instruments, number_of_steps=5)
forecasted_returns = forc.forecast_returns()

 [0.07966349 0.83541521 0.09930475] True
 [0.06109371 0.79387629 0.18175313] False
 [0.02161618 0.68328501 0.43294249] False
 [0.04267142 0.84278157 0.19959429] False
 [0.02231843 0.85344123 0.09842483] True


In [4]:
for row in forecasted_returns:
    print(row*100)

[ 0.17522024 -0.04226988  0.02684653 -0.03002656 -0.13309249]
[ 0.09499433  0.13167483 -0.05028183 -0.1044066  -0.33538317]
[ 0.01780704  0.08921025  0.01222081 -0.01708068 -0.19707606]
[-0.32831414  0.07129041  0.16417404  0.13764769 -0.25271772]
[ 0.03196275  0.01865388 -0.01087417 -0.06332288 -0.12466083]


In [5]:
cash_returns = np.zeros(5)
cash_returns

array([0., 0., 0., 0., 0.])

In [6]:
forecasted_returns = np.vstack([cash_returns, forecasted_returns])

In [7]:
for row in forecasted_returns:
    print(np.round(row*100, 2))

[0. 0. 0. 0. 0.]
[ 0.18 -0.04  0.03 -0.03 -0.13]
[ 0.09  0.13 -0.05 -0.1  -0.34]
[ 0.02  0.09  0.01 -0.02 -0.2 ]
[-0.33  0.07  0.16  0.14 -0.25]
[ 0.03  0.02 -0.01 -0.06 -0.12]


In [8]:
number_of_assets = 6
number_of_steps = 5

In [14]:
Z = cp.Variable((number_of_assets, number_of_steps))

R = forecasted_returns
ones = np.ones(number_of_assets)
init_portfolio = np.array([1, 0, 0, 0, 0, 0])

cost = 0
constr = []

for t in range(number_of_steps):
    cost += R[:,t].T @ Z[:,t]
    constr += [Z[:,t] <= 1, Z[:,t] >= 0]


In [15]:
problem = cp.Problem(cp.Maximize(cost), constr)

In [16]:
problem.solve(verbose=True)

                                     CVXPY                                     
                                     v1.2.1                                    
(CVXPY) Aug 21 04:22:30 PM: Your problem has 30 variables, 10 constraints, and 0 parameters.
(CVXPY) Aug 21 04:22:30 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Aug 21 04:22:30 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Aug 21 04:22:30 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Aug 21 04:22:30 PM: Compiling problem (target solver=ECOS).
(CVXPY) Aug 21 04:22:30 PM: Reduction chain: FlipObjective -> Dcp2Cone -> CvxAttr2Constr -> Con

0.00971702693688958

In [17]:
print(Z.value)

[[5.00005629e-01 5.00005629e-01 5.00005629e-01 5.00005629e-01
  5.00005629e-01]
 [9.99999978e-01 8.75522753e-08 9.99999861e-01 1.21112798e-07
  3.33891061e-08]
 [9.99999949e-01 9.99999966e-01 8.03892422e-08 4.60525991e-08
  1.20483603e-08]
 [9.99999783e-01 9.99999945e-01 9.99999683e-01 2.26207066e-07
  1.80216062e-08]
 [1.20464812e-08 9.99999933e-01 9.99999976e-01 9.99999968e-01
  1.32203010e-08]
 [9.99999888e-01 9.99999793e-01 3.15909839e-07 7.23060930e-08
  3.66203584e-08]]


In [13]:
def power_utility(x: float, gamma: float) -> float:
    """Constant RRA utility function. Optimal for investments (source Jorgen).

    Args:
        x (float): variable value
        gamma (float): risk aversion parameter

    Raises:
        ValueError: if gamma is greater than 1.

    Returns:
        float: utility of x.
    """

    if gamma > 1:
        raise ValueError("Value of gamma is larger than 1.")
    if gamma != 0:
        return x ** gamma / gamma
    else:
        return np.log(x)